In [1]:
import unicodedata
import re
import os
import json
import requests
from bs4 import BeautifulSoup 

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import acquire as a

In [2]:
##### Imports #####
import unicodedata
import re
import os
import json
import requests
from bs4 import BeautifulSoup 

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import acquire as a


##### Funcitons #####

def basic_clean(string):
    article = string.lower()
    article = unicodedata.normalize('NFKD', article).encode('ascii', 'ignore').decode('utf-8')
    #article = re.sub(r"[^\w]", ' ', article)
    article = re.sub("[^a-z0-9'\s]", '', article)
    article = re.sub("\\n", ' ', article)

    return article


def tokenize(string):
    article = []
    # Create the tokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()

    # Use the tokenizer
    article = tokenizer.tokenize(string, return_str = True)

    return article


def stem(string):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in string.split()]
    article_stemmed = ' '.join(stems)
    
    return article_stemmed


def lemmatize(text):
    # Create the Lemmatizer.
    wnl = nltk.stem.WordNetLemmatizer()
    # use the lemmatizer
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    # Join the lemmatized words together
    article_lemmatized = ' '.join(lemmas)
    
    return article_lemmatized


def remove_stopwords(string, extra_words=[], exclude_words=[]):   
    stopword_list = stopwords.words('english')
    # Remove 'exclude_words' from stopword_list to keep these in my text.
    stopword_list = set(stopword_list) - set(exclude_words)
    # Add in 'extra_words' to stopword_list.
    stopword_list = stopword_list.union(set(extra_words))
    # Split words in lemmatized article.
    words = string.split()
    # Iterate through the words and only keep those not in stopword_list
    filtered_words = [word for word in words if word not in stopword_list]
    # Join words in the list back into strings; assign to a variable to keep changes.
    article_without_stopwords = ' '.join(filtered_words)

    return article_without_stopwords

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasontellez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasontellez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_json('data.json')
df

,repo,language,readme_contents
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <..."
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...
...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...


In [43]:
df['clean'] = df['readme_contents'].apply(basic_clean).apply(tokenize).apply(lemmatize).apply(remove_stopwords, extra_words=['build', 'file', 'example', 'code', 'use', 'using'], exclude_words=[])
df

,repo,language,readme_contents,clean
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <...",h1 aligncenter react native window h1 p alignc...
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...,azure monitor workbook template statushttpsgit...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...,microsoft gctoolkit gctoolkit set library anal...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...,welcome wingetclirestsource repository buildin...
...,...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...,multicluster multitenant environment flux v2 r...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...,kubeflow pipeline event handler microservice e...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...,builddockerprovider superproject dockerprovide...


In [44]:
df.clean = df.clean.str.strip()
df

,repo,language,readme_contents,clean
0,microsoft/react-native-windows,C++,"<h1 align=""center""> React Native for Windows <...",h1 aligncenter react native window h1 p alignc...
1,microsoft/fast,TypeScript,\n![fast_banner_github_914.png](https://static...,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,# Azure Monitor Workbook Templates [![Build St...,azure monitor workbook template statushttpsgit...
3,microsoft/gctoolkit,Java,# Microsoft GCToolKit\n\nGCToolkit is a set of...,microsoft gctoolkit gctoolkit set library anal...
4,microsoft/winget-cli-restsource,C#,# Welcome to the winget-cli-restsource reposit...,welcome wingetclirestsource repository buildin...
...,...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,[![](https://img.shields.io/badge/docs-stable-...,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1496,microsoft/multicluster-gitops,Shell,# Multi-cluster and multi-tenant environment w...,multicluster multitenant environment flux v2 r...
1497,microsoft/kfp-event-handler,Go,# Kubeflow Pipelines Event Handler\n\nThis mic...,kubeflow pipeline event handler microservice e...
1498,microsoft/Build-Docker-Provider,Shell,# Build-Docker-Provider\nSuperproject for Dock...,builddockerprovider superproject dockerprovide...


In [46]:
df.loc[pd.Series(' '.join(df.clean.astype('str')).split()).isin(pd.Series('using'))]

,repo,language,readme_contents,clean


In [47]:
pd.Series(' '.join(df.clean.astype('str')).split()).isin(pd.Series('example')).mean()

0.0

In [49]:
df.dropna(subset=['language'], inplace=True)

In [50]:
df.clean.isnull().any()

False

In [51]:
len(df)

1376

In [52]:
df = df[['repo', 'language', 'clean']]

In [53]:
df = df.rename(columns={"lemmatized": "content"})

In [54]:
df

,repo,language,clean
0,microsoft/react-native-windows,C++,h1 aligncenter react native window h1 p alignc...
1,microsoft/fast,TypeScript,fastbannergithub914pnghttpsstaticfastdesignass...
2,microsoft/Application-Insights-Workbooks,JSON,azure monitor workbook template statushttpsgit...
3,microsoft/gctoolkit,Java,microsoft gctoolkit gctoolkit set library anal...
4,microsoft/winget-cli-restsource,C#,welcome wingetclirestsource repository buildin...
...,...,...,...
1495,microsoft/AzureClusterlessHPC.jl,Julia,httpsimgshieldsiobadgedocsstablebluesvghttpsmi...
1496,microsoft/multicluster-gitops,Shell,multicluster multitenant environment flux v2 r...
1497,microsoft/kfp-event-handler,Go,kubeflow pipeline event handler microservice e...
1498,microsoft/Build-Docker-Provider,Shell,builddockerprovider superproject dockerprovide...


In [ ]:
df = df.reset_index(drop=True)

df.to_csv('cleaned_readmes.csv')

In [ ]:
stopword_list = stopwords.words('english')
stopword_list.append()

In [ ]:
df

In [13]:
pd.Series(' '.join(df.clean.astype('str')).split()).isin(pd.Series('using')).mean()

0.005464758984407306